# 21. 函数闭包的作用域相关注意事项

python支持闭包，定义在大函数里的小函数可以引用大函数作用域内的变量，函数在python中是头等对象

在表达式中引用某个变量时，Python解释器会按照下面的顺序，在各个作用域（scope）里面查找这个变量，以解析（resolve）这次引用[1]。
1）当前函数的作用域。
2）外围作用域（例如包含当前函数的其他函数所对应的作用域）。
3）包含当前代码的那个模块所对应的作用域（也叫全局作用域，global scope）。
4）内置作用域（built-in scope，也就是包含len与str等函数的那个作用域）。
如果这些作用域中都没有定义名称相符的变量，那么程序就抛出NameError异常


python变量的赋值要注意，如果变量已经定义在当前作用域中，那么直接把新值交给它就行了。如果当前作用域中不存在这个变量，那么即便外围作用域里有同名的变量，Python也还是会把这次的赋值操作当成变量的定义来处理，这会产生一个重要的效果，也就是说，Python会把包含赋值操作的这个函数当成新定义的这个变量的作用域

In [2]:
numbers = [8,3,2,9,6,5,4,99]
group = [8,99,2]
def sort_priority(values,group):
    found = False
    def helper(x):
        if x in group:
            found = True # 实际上，这里的found相当于新变量来定义，并不会传递给上层的found
            return (0 ,x)
        return (1,x)
    numbers.sort(key=helper)
    return found

found = sort_priority(numbers,group)
print(numbers)
print(f'{found=}') # 返回结果还是False

[2, 8, 99, 3, 4, 5, 6, 9]
found=False


Python有一种特殊的写法，可以把闭包里面的数据赋给闭包外面的变量。 nonlocal语句描述变量，可以让系统在处理针对这个变量的赋值操作时，去外围作用域查找。有一种跟它互补的语句，叫作global，用这种语句描述某个变量后，在给这个变量赋值时，系统会直接把它放到模块作用域（或者说全局作用域）中

In [4]:
def sort_priority(values,group):
    found = False
    def helper(x):
        nonlocal found
        if x in group:
            found = True # 实际上，这里的found相当于新变量来定义，并不会传递给上层的found
            return (0 ,x)
        return (1,x)
    numbers.sort(key=helper)
    return found

found = sort_priority(numbers,group)
print(f'{found=}') # 返回结果为True

found=True


nonlocal语句并不是推荐经常使用，因为其造成的副作用有时很难发现，更推荐下面的辅助类方法，更加清晰

In [6]:
class Sorter:
    def __init__(self,group):
        self.found = False
        self.group=group
    def __call__(self,x):
        if x in group:
            self.found = True
            return (0,x)
        return (1,x)
    
sorter = Sorter(group)
numbers.sort(key=sorter)
print(f'{numbers=},{sorter.found}')

numbers=[2, 8, 99, 3, 4, 5, 6, 9],True


# 22. 可变位置参数*args的用法

程序总是必须先把这些参数转化成一个元组，然后才能把它们当成可选的位置参数传给函数。这意味着，如果调用函数时，把带*操作符的生成器传了过去，那么程序必须先把这个生成器里的所有元素迭代完（以便形成元组），然后才能继续往下执行（相关知识，参见第30条）。这个元组包含生成器所给出的每个值，这可能耗费大量内存，甚至会让程序崩溃。

*args参数的函数，适合处理输入值不太多，而且数量可以提前预估的情况

In [7]:
def tmp_generator():
    for i in range(10):
        yield i

def my_func(*args):
    print(args)

it = tmp_generator()
my_func(*it) #输出是元组3
'''
*操作符会让Python把序列中的元素都当成位置参数传给这个函数
'''

(0, 1, 2, 3, 4, 5, 6, 7, 8, 9)


# 25. 只能以关键字指定和只能以位置参数指定

/符号左边的参数只能以位置参数指定，*符号右边的参数，只能以关键字参数指定，/和*之间的参数，二者都可以，这也是函数默认参数的规则，参考下面的例子

In [9]:
def safe_division(number,divisor,/,*,ignore_overflow=False,ignore_zero_division=True):
    try:
        return number/divisor
    except OverflowError:
        if ignore_overflow:
            print("ignore overflowerror")
            return 0
        else:
            raise
    except ZeroDivisionError:
        if ignore_zero_division:
            return float('inf')
        else:
            raise

In [12]:
safe_division(1,2,ignore_overflow=True,ignore_zero_division=False)

0.5

# 26. 用functools.wraps定义函数修饰器

In [13]:
from functools import wraps

# 定义一个修饰器，并且用functools.wraps来保存内层函数的元信息到外面
def trace(func):
    @wraps(func)
    def wrapper(*args,**kwargs):
        result = func(*args,**kwargs)
        print(f'{func.__name__}({args!r},{kwargs!r})')
        return result
    return wrapper

@trace
def fibonacci(n):
    if n in (0,1):
        return 1
    return fibonacci(n-1)+fibonacci(n-2)

fibonacci(10)

fibonacci((1,),{})
fibonacci((0,),{})
fibonacci((2,),{})
fibonacci((1,),{})
fibonacci((3,),{})
fibonacci((1,),{})
fibonacci((0,),{})
fibonacci((2,),{})
fibonacci((4,),{})
fibonacci((1,),{})
fibonacci((0,),{})
fibonacci((2,),{})
fibonacci((1,),{})
fibonacci((3,),{})
fibonacci((5,),{})
fibonacci((1,),{})
fibonacci((0,),{})
fibonacci((2,),{})
fibonacci((1,),{})
fibonacci((3,),{})
fibonacci((1,),{})
fibonacci((0,),{})
fibonacci((2,),{})
fibonacci((4,),{})
fibonacci((6,),{})
fibonacci((1,),{})
fibonacci((0,),{})
fibonacci((2,),{})
fibonacci((1,),{})
fibonacci((3,),{})
fibonacci((1,),{})
fibonacci((0,),{})
fibonacci((2,),{})
fibonacci((4,),{})
fibonacci((1,),{})
fibonacci((0,),{})
fibonacci((2,),{})
fibonacci((1,),{})
fibonacci((3,),{})
fibonacci((5,),{})
fibonacci((7,),{})
fibonacci((1,),{})
fibonacci((0,),{})
fibonacci((2,),{})
fibonacci((1,),{})
fibonacci((3,),{})
fibonacci((1,),{})
fibonacci((0,),{})
fibonacci((2,),{})
fibonacci((4,),{})
fibonacci((1,),{})
fibonacci((0,),{})
fibonacci((2

89